In [241]:
import pandas as pd
import cohere as co
from neo4j import GraphDatabase as gdb
import weaviate as wev
import weaviate.classes.config as wc

In [242]:
movie_data = pd.read_csv('movie_data.csv', engine='python')


In [243]:
movie_data.head()

,id,title,overview,genres,cast,poster_url,release_date,popularity,vote_average,vote_count,original_language,adult
0,533535,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,"['Action', 'Comedy', 'Science Fiction']","['Ryan Reynolds', 'Hugh Jackman', 'Emma Corrin...",https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUT...,2024-07-24,9866.173,7.776,2074,en,False
1,718821,Twisters,"As storm season intensifies, the paths of form...","['Action', 'Adventure', 'Drama', 'Thriller']","['Daisy Edgar-Jones', 'Glen Powell', 'Anthony ...",https://image.tmdb.org/t/p/w500/pjnD08FlMAIXsf...,2024-07-10,4687.952,7.07,865,en,False
2,519182,Despicable Me 4,"Gru and Lucy and their girls—Margo, Edith and ...","['Animation', 'Family', 'Comedy', 'Action']","['Steve Carell', 'Kristen Wiig', 'Will Ferrell...",https://image.tmdb.org/t/p/w500/wWba3TaojhK7Nd...,2024-06-20,3270.437,7.327,1104,en,False
3,1022789,Inside Out 2,Teenager Riley's mind headquarters is undergoi...,"['Animation', 'Family', 'Adventure', 'Comedy']","['Amy Poehler', 'Maya Hawke', 'Kensington Tall...",https://image.tmdb.org/t/p/w500/vpnVM9B6NMmQpW...,2024-06-11,3210.402,7.604,2438,en,False
4,573435,Bad Boys: Ride or Die,"After their late former Captain is framed, Low...","['Action', 'Crime', 'Thriller', 'Comedy']","['Will Smith', 'Martin Lawrence', 'Vanessa Hud...",https://image.tmdb.org/t/p/w500/oGythE98MYleE6...,2024-06-05,2973.442,7.62,1610,en,False


In [244]:
movie_data.isnull().sum()

id                    0
title                 0
overview             76
genres                1
cast                  1
poster_url            1
release_date         36
popularity            1
vote_average          1
vote_count            1
original_language     2
adult                 2
dtype: int64

In [245]:
movie_data.duplicated().sum()

np.int64(0)

In [246]:
movie_data.dropna(inplace=True)

In [247]:
movie_data.isnull().sum()

id                   0
title                0
overview             0
genres               0
cast                 0
poster_url           0
release_date         0
popularity           0
vote_average         0
vote_count           0
original_language    0
adult                0
dtype: int64

In [248]:
movie_data

,id,title,overview,genres,cast,poster_url,release_date,popularity,vote_average,vote_count,original_language,adult
0,533535,Deadpool & Wolverine,A listless Wade Wilson toils away in civilian ...,"['Action', 'Comedy', 'Science Fiction']","['Ryan Reynolds', 'Hugh Jackman', 'Emma Corrin...",https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUT...,2024-07-24,9866.173,7.776,2074,en,False
1,718821,Twisters,"As storm season intensifies, the paths of form...","['Action', 'Adventure', 'Drama', 'Thriller']","['Daisy Edgar-Jones', 'Glen Powell', 'Anthony ...",https://image.tmdb.org/t/p/w500/pjnD08FlMAIXsf...,2024-07-10,4687.952,7.07,865,en,False
2,519182,Despicable Me 4,"Gru and Lucy and their girls—Margo, Edith and ...","['Animation', 'Family', 'Comedy', 'Action']","['Steve Carell', 'Kristen Wiig', 'Will Ferrell...",https://image.tmdb.org/t/p/w500/wWba3TaojhK7Nd...,2024-06-20,3270.437,7.327,1104,en,False
3,1022789,Inside Out 2,Teenager Riley's mind headquarters is undergoi...,"['Animation', 'Family', 'Adventure', 'Comedy']","['Amy Poehler', 'Maya Hawke', 'Kensington Tall...",https://image.tmdb.org/t/p/w500/vpnVM9B6NMmQpW...,2024-06-11,3210.402,7.604,2438,en,False
4,573435,Bad Boys: Ride or Die,"After their late former Captain is framed, Low...","['Action', 'Crime', 'Thriller', 'Comedy']","['Will Smith', 'Martin Lawrence', 'Vanessa Hud...",https://image.tmdb.org/t/p/w500/oGythE98MYleE6...,2024-06-05,2973.442,7.62,1610,en,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9940,7348,Robinson Crusoe on Mars,Stranded on Mars with only a monkey as a compa...,"['Science Fiction', 'Adventure']","['Paul Mantee', 'Victor Lundin', 'Adam West', ...",https://image.tmdb.org/t/p/w500/kzs1HUxP1HzLNd...,1964-06-01,19.936,6.263,133,en,False
9942,29427,The Crazies,Four friends find themselves trapped in their ...,"['Mystery', 'Horror', 'Action']","['Timothy Olyphant', 'Radha Mitchell', 'Joe An...",https://image.tmdb.org/t/p/w500/ggtG9DZELyTtM2...,2010-02-26,19.932,6.263,1989,en,False
9943,774741,Diary of a Wimpy Kid,Greg Heffley is a scrawny but ambitious kid wi...,"['Animation', 'Comedy', 'Family']","['Brady Noon', 'Ethan William Childress', 'Hun...",https://image.tmdb.org/t/p/w500/9ygyYYbBtBRaAn...,2021-12-02,19.929,6.179,344,en,False
9944,25941,Harry Brown,An elderly ex-serviceman and widower looks to ...,"['Thriller', 'Crime', 'Drama', 'Action']","['Michael Caine', 'Emily Mortimer', 'Iain Glen...",https://image.tmdb.org/t/p/w500/lk9qVzvPHuL2vX...,2009-11-11,19.928,6.8,1021,en,False


In [249]:
movie_data = movie_data[['title', 'overview', 'genres', 'cast', 'poster_url', 'release_date']]

## Load csv data and populate  Neo4j

### Function to create a Neo4j driver

In [250]:
def create_driver(uri, username, password):
    driver = gdb.driver(uri, auth=(username, password))
    return driver

### Function to insert movies into the Neo4j database

In [251]:
def insert_movie(tx, movie):
    # cypher query to create a Movie node with it's properties
    query = """
    MERGE (m:Movie {title: $title})
    SET m.overview = $overview,
        m.release_date = $release_date,
        m.poster_url = $poster_url
    """
    tx.run(query, movie)

### Function to insert genres and their relationships with movies

In [252]:
def insert_genres(tx, title, genres):
    for genre in genres:
        query = """
            MERGE (g:Genre {name: $genre})
            WITH g
            MATCH (m:Movie {title: $title})
            MERGE (m)-[:BELONGS_TO]->(g)
        """
        tx.run(query, genre=genre, title=title)

### Function to insert cast members and their relationships with movies

In [253]:
def insert_cast(tx, title, cast):
    for actor in cast:
        query = """
            MERGE (p:Person {name: $actor})
            WITH p
            MATCH (m:Movie {title: $title})
            MERGE (p)-[:ACTED_IN]->(m)
        """
        tx.run(query, actor=actor, title=title)

### Function to populate the Neo4j database with movie data from a CSV

In [254]:
def populate_neo4j(uri, username, password, csv_file):
    # Load csv file
    df = pd.read_csv(csv_file, engine='python')

    # Create the Neo4j driver
    driver = create_driver(uri, username, password)

    # Use a session to execute write transactions
    with driver.session() as session:
        for index, row in df.iterrows():
            movie = {
                'title': row['title'],
                'overview': row['overview'],
                'release_date': row['release_date'],
                'poster_url': row['poster_url']
            }
            genres = eval(row['genres']) if isinstance(row['genres'], str) else []
            cast = eval(row['cast']) if isinstance(row['cast'], str) else []

            # Insert Movie node
            session.write_transaction(insert_movie, movie)

            # Insert genre relationship
            session.write_transaction(insert_genres, movie['title'], genres)

            # Insert Cast relationship
            session.write_transaction(insert_cast, movie['title'], cast)

    driver.close()

In [255]:
from dotenv import load_dotenv
import os

load_dotenv()

uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
csv_file = 'movies.csv'

### Testing the connection to the neo4j password

In [256]:

def test_connection(uri, username, password):
    driver = gdb.driver(uri, auth=(username, password))
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Connection successful!'")
            for record in result:
                print(record)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

# Test connection
test_connection(uri, username, password)

<Record 'Connection successful!'='Connection successful!'>


### Calling the populate_neo4j function

In [257]:
# populate_neo4j(uri, username, password, csv_file)

### Querying for Movie Recommendations

In [258]:
from neo4j import GraphDatabase

def create_driver(uri, username, password):
    return GraphDatabase.driver(uri, auth=(username, password))

def close_driver(driver):
    driver.close()

In [259]:
# Retrieve movies similar to a given movie based on shared genres
def get_movie_recommendations(tx, title, limit=5):
    query = """
    MATCH (m:Movie {title: $title})-[:BELONGS_TO]->(g:Genre)<-[:BELONGS_TO]-(rec:Movie)
    WHERE m <> rec
    RETURN rec.title AS title, COUNT(g) AS genre_overlap
    ORDER BY genre_overlap DESC
    LIMIT $limit
    """
    result = tx.run(query, title=title, limit=limit)
    return [record['title'] for record in result]


In [260]:
def recommend_movies(uri, username, password, movie_title, limit=5):
    driver = create_driver(uri, username, password)
    recommendations = []

    with driver.session() as session:
        recommendations = session.read_transaction(get_movie_recommendations, movie_title, limit)
    close_driver(driver)
    return recommendations

In [261]:
from dotenv import load_dotenv
import os

load_dotenv()

uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

In [262]:
movie_title = "Inside Out 2"
recommendations = recommend_movies(uri, username, password, movie_title, limit=5)

/var/folders/vk/tmm1dhjx7p30tdfcvdz9_m280000gn/T/ipykernel_13466/1450798905.py:6: DeprecationWarning: read_transaction has been renamed to execute_read
  recommendations = session.read_transaction(get_movie_recommendations, movie_title, limit)


In [263]:
for i, movie in enumerate(recommendations, start=1):
    print(f'{i}. {movie}')

1. Migration
2. Inside Out
3. Saving Bikini Bottom: The Sandy Cheeks Movie
4. The Garfield Movie
5. Run, Tiger Run!


### Weaviate Database

In [264]:
weaviate_url = os.getenv('WCD_URL')
weaviate_api_key = os.getenv('API_KEY')
huggingFace_access_token = os.getenv('HuggingFace_AccessToken')

In [265]:
client = wev.connect_to_wcs(
    cluster_url=weaviate_url,
    auth_credentials=wev.auth.AuthApiKey(weaviate_api_key),
    headers={'X-Huggingface-Api-Key': huggingFace_access_token},
    skip_init_checks=True
    )

client.is_ready()

True

### Create a collection for movies

In [266]:
# Define the movie properties and configurations
movie_properties = [
    wc.Property(name="title", data_type=wc.DataType.TEXT),
    wc.Property(name="overview", data_type=wc.DataType.TEXT)
]

vectorizer_config = wc.Configure.Vectorizer.text2vec_huggingface()

vector_index_config = wc.Configure.VectorIndex.hnsw(
    distance_metric=wc.VectorDistances.COSINE
)

generative_config = wc.Configure.Generative.openai()

references = [
    wc.ReferenceProperty(name="hasReview", target_collection="Review")
]

if not client.collections.exists('Movie'):
    client.collections.create(
        name='Movie',
        properties=movie_properties,
        vectorizer_config=vectorizer_config,
        vector_index_config=vector_index_config,
        generative_config=generative_config,
        # references=references
    )
    print('Collection for Movie class created successfully.')
else:
    print('Collection for Movie class already exists.')



Collection for Movie class already exists.


### Add data to the Movie collection

In [267]:
movies = client.collections.get('Movie')

#### 1. Single Insert

In [268]:
movies.data.insert(
    properties={'title': 'Twisters', 'overview': 'As storm season intensifies, the paths of form...'}
)

UUID('cbb30f06-9b1d-43c2-836a-921486b97048')